# 🔬 Vectorius — Azure OpenAI Proof (Lesson 15)

This notebook connects to your **Azure OpenAI** deployment and loads **Markdown prompts** from `/prompts` so you can edit pedagogy without touching code.

### What this does
1. Load credentials from `web/.env.local`
2. Read `prompts/grade8_system.md` + a mode file (`tutor | checker | explainer`)
3. Initialize Azure OpenAI client
4. Send a test chat completion using the same prompt stack as the web app

In [1]:
import os
from dotenv import load_dotenv

# Load variables from web/.env.local (run from /experiments or similar)
ok = load_dotenv(dotenv_path="../web/.env.local")
ok

True

## 1) Verify environment variables
We expect **all three** below to be set in `web/.env.local`:

```ini
AZURE_OPENAI_ENDPOINT=https://<your-resource>.openai.azure.com/
AZURE_OPENAI_API_KEY=***
AZURE_OPENAI_DEPLOYMENT=<your-deployment-name>
```

In [2]:
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("Endpoint:", endpoint)
print("Deployment:", deployment)
print("Key loaded:", "✔️" if api_key else "❌")

Endpoint: https://jaapd-meryoqet-eastus2.cognitiveservices.azure.com/
Deployment: gpt-4.1-mini-vectorius
Key loaded: ✔️


## 2) Load Markdown prompts (shared with the web app)
Prompts live in the repo root under `/prompts`. We read them via the small helper in `/experiments/prompt_loader.py`.

In [3]:
# Make sure we can import experiments/prompt_loader.py from anywhere
# import sys, pathlib

# here = pathlib.Path.cwd()
# for p in [here, *here.parents]:
#     exp = p / "experiments"
#     if (exp / "prompt_loader.py").exists():
#         sys.path.insert(0, str(exp))   # add /experiments to import path
#         break

from prompt_loader import read_prompt  # <-- now this works

system_text = read_prompt("grade8_system.md")
tutor_text  = read_prompt("tutor_mode.md")
checker_text = read_prompt("checker_mode.md")
explainer_text = read_prompt("explainer_mode.md")

print("System prompt length:", len(system_text))
print("Tutor/Checker/Explainer lengths:", len(tutor_text), len(checker_text), len(explainer_text))

System prompt length: 2215
Tutor/Checker/Explainer lengths: 735 657 690


In [4]:
import pathlib
print("CWD:", pathlib.Path.cwd())
print("read_prompt OK:", len(read_prompt("grade8_system.md")) > 0)


CWD: c:\Users\Innovation4\Documents\Vectorius\vectorius-prototype-next\experiments
read_prompt OK: True


## 3) Initialize Azure OpenAI client
Uses the official `openai` Python SDK in Azure mode.

In [5]:
from openai import AzureOpenAI

api_version = "2024-12-01-preview"  # adjust to your Azure portal version

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=endpoint
)
print("Client ready ✔️")

Client ready ✔️


## 4) Helpers to build messages with mode
Mode can be one of: `tutor`, `checker`, or `explainer`.

In [6]:
def messages_with_mode(user_text: str, mode: str = "tutor"):
    mode = (mode or "tutor").lower().strip()
    if mode == "checker":
        mode_text = checker_text
    elif mode == "explainer":
        mode_text = explainer_text
    else:
        mode_text = tutor_text

    return [
        {"role": "system", "content": system_text},
        {"role": "system", "content": mode_text},
        {"role": "user", "content": user_text},
    ]

def chat_once(user_text: str, mode: str = "tutor", temperature: float = 0.4):
    msgs = messages_with_mode(user_text, mode)
    resp = client.chat.completions.create(
        model=deployment,
        messages=msgs,
        temperature=temperature,
    )
    return resp.choices[0].message.content

## 5) Tests
- Tutor mode → asks for attempt, gives hint
- Checker mode → rubric + fixes + next step
- Explainer mode → short concept + different example

In [10]:
print("Tutor mode:")
print(chat_once("Who was king tut?", mode="tutor"))

print("\nChecker mode:")
student_attempt = "I know he was a farao in Egypt and he lived around 1000 BC."
print(chat_once(f"PLease check my asnwer: {student_attempt}", mode="checker"))

print("\nExplainer mode:")
print(chat_once("Tell me more about King Uts life.", mode="explainer"))

Tutor mode:
Hi! What do you already know about King Tut?  
If you’re not sure, that’s okay!  

- **Hint:** Start by finding out when King Tut lived and why he is famous.  
- **Why it helps:** Knowing the time period and his significance gives a clear picture of who he was.  
- **Your turn:** Can you try to write one or two sentences about when King Tut lived or what he is known for?

Checker mode:
- **Rubric:**  
  1. Accuracy of historical facts  
  2. Clarity and completeness of information  

- **Top fixes:**  
  1. "Farao" should be spelled "pharaoh."  
  2. The date "around 1000 BC" might need to be more precise or supported with context (which pharaoh?).

- **Next step:**  
  Check your facts about which pharaoh you mean and add one or two key details about their reign or significance.

Explainer mode:
Could you tell me which King Uts you mean? There are different kings with similar names in history. If you mean a specific one, like from a certain country or time period, I can he

## Testing the CURL Command

In [12]:
import requests
import json

# Local Next.js server URL
url = "http://localhost:3000/api/chat"

payload = {
    "question": "help me solve 2x+5=17",
    "mode": "tutor"
}

headers = {"Content-Type": "application/json"}

resp = requests.post(url, headers=headers, data=json.dumps(payload))

print("Status:", resp.status_code)
print("Response:", resp.json())


Status: 200
Response: {'reply': 'Hi! What have you tried so far with the equation 2x + 5 = 17?'}
